# This notebook serves to create a .db file that contains the tables used in the Datacamp course Joining Data in SQL. My environment was problematic using other flavors of SQL (such as MySQL) so I decided to go through Python. This is not the most eligant, or automated process. It serves only as a quick adhoc way to create a database with tables that can be queried using sqlite3 in python. We also make use of Pandas to visualize our queries

In [1]:
import re
import pandas as pd
import sqlite3
from sqlite3 import OperationalError

In [2]:
#This function will strip our commands in the countries.sql file and keep important ones to make database
def createcommand(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')
    
    test = []
    for command in sqlCommands:
        test.append(re.sub(' +', ' ',command.replace('\n',''))+';')
        
    sqlCommands = test
    return sqlCommands

In [3]:
#Check list of commands for table creation, as \copy throws error in sqlite3 using python
test_command = createcommand('countries.sql')
print(test_command[:9])

['CREATE TABLE cities ( name VARCHAR PRIMARY KEY, country_code VARCHAR, city_proper_pop REAL, metroarea_pop REAL, urbanarea_pop REAL);', 'CREATE TABLE countries ( code VARCHAR PRIMARY KEY, name VARCHAR, continent VARCHAR, region VARCHAR, surface_area REAL, indep_year INTEGER, local_name VARCHAR, gov_form VARCHAR, capital VARCHAR, cap_long REAL, cap_lat REAL);', 'CREATE TABLE languages ( lang_id INTEGER PRIMARY KEY, code VARCHAR, name VARCHAR, percent REAL, official BOOLEAN);', 'CREATE TABLE economies ( econ_id INTEGER PRIMARY KEY, code VARCHAR, year INTEGER, income_group VARCHAR, gdp_percapita REAL, gross_savings REAL, inflation_rate REAL, total_investment REAL, unemployment_rate REAL, exports REAL, imports REAL);', 'CREATE TABLE currencies ( curr_id INTEGER PRIMARY KEY, code VARCHAR, basic_unit VARCHAR, curr_code VARCHAR, frac_unit VARCHAR, frac_perbasic REAL);', 'CREATE TABLE populations ( pop_id INTEGER PRIMARY KEY, country_code VARCHAR, year INTEGER, fertility_rate REAL, life_expec

In [4]:
#This function will be the driver for calling our commands and creating the tables
def builddb(query,db_file):
    for item in query:
        try:
            conn = sqlite3.connect(db_file)
            c = conn.cursor()
            c.execute(item)
        except OperationalError:
            print(Error)

In [5]:
#Make the database
# countries_command = test_command[:9]
countries_command = test_command[:9]
builddb(countries_command,'countries.db')

In [6]:
#Check that we have created tables successfully

#This short functions alows us to use pandas to view the tables using sql commands
def query_to_df (query):
    return pd.read_sql_query(query,db)

In [7]:
#Set the database file
db = sqlite3.connect('countries.db')

In [8]:
#Check that all the desired tables were created
query = "SELECT name FROM sqlite_master WHERE type='table';"
query_to_df(query)

,name
0,cities
1,countries
2,languages
3,economies
4,currencies
5,populations
6,countries_plus
7,economies2010
8,economies2015


In [20]:
#Check specific table column names, it will be important to match these to the csv files used later
query = "SELECT * FROM populations;"
query_to_df(query)

,pop_id,country_code,year,fertility_rate,life_expectancy,size


In [21]:
#Now we need to populate the tables using our csv files

#Create list of csv files to add to tables
csv_list = ['cities.csv','countries.csv','languages.csv','economies.csv','economies2010.csv','economies2015.csv','currencies.csv','populations.csv','countries_plus.csv']

for entry in csv_list:
    #Import csv as dataframe
    df = pd.read_csv(entry)
    
    #here from above we are still connected to db for our pandas tool, so we make use of it
    
    #set the query to make a dataframe object for the specific table in loop
    query = 'SELECT * FROM {}'.format(entry.replace('.csv',''))
    #Give it a name
    table_df = query_to_df(query)
    #Set the name of the df columns so we do not get an error in to_sql call
    df.columns = table_df.columns
    #Finally import the dataframe into the database
    df.to_sql(entry.replace('.csv',''), db, if_exists='append', index=False)

In [32]:
#Check that we have success
query = "SELECT * FROM economies2015;"
query_to_df(query)

,code,year,income_group,gross_savings
0,AFG,2015,Low income,21.466
1,AGO,2015,Upper middle income,-0.425
2,ALB,2015,Upper middle income,13.840
3,ARE,2015,High income,34.106
4,ARG,2015,Upper middle income,14.111
...,...,...,...,...
185,WSM,2015,Lower middle income,NaN
186,YEM,2015,Lower middle income,-3.715
187,ZAF,2015,Upper middle income,16.460
188,ZMB,2015,Lower middle income,39.177


In [23]:
for entry in csv_list:
    #Import csv as dataframe
    df = pd.read_csv(entry)
    print(df.shape)

(236, 5)
(206, 11)
(955, 5)
(380, 11)
(190, 4)
(190, 4)
(224, 6)
(434, 6)
(206, 5)


In [33]:
#All sizes match so we have a working database